In [29]:
import numpy as np
import pandas as pd
import os
from pycbc.io import HFile, get_all_subkeys # hfile basicamente lee HDF y get da todas las claves en forma de lista
from pycbc.events import coinc # esto dice en la documentación: This module contains functions for calculating and manipulating coincident triggers
import pycbc.io #esto trae todo lo de pycbc

In [19]:
def get_trigger_info(ttype, sfile, bank, merge_trig_temp):
    '''Read all relevant information
    #ttype: El tipo de trigger que se desea analizar
    #sfile: Un archivo de datos en un formato HDF 
    #bank: Un banco de plantillas que contiene información sobre la configuración de eventos simulados 
    #merge_trig_temp: Plantilla de archivo para los datos de triggers individuales.
    '''
    merge_fields = ['coa_phase', 'snr']
    #ifos toma el atributo ifos del archivo sfile y lo divide en una lista de interferómetros, que se usan para acceder a datos específicos de cada detector
    ifos = sfile.attrs['ifos'].split(' ') 
    
    params = {}
    params['stat'] = sfile[f'{ttype}/stat'.format(section)][:]  #esto tiene longitud 2060
    if 'ifar' in sfile[f'{ttype}'.format(section)]:
        params['ifar'] = sfile[f'{ttype}/ifar'.format(section)][:] #esto tambien tiene longitud 2060
    elif 'ifar_exc' in sfile[f'{ttype}'.format(section)]:
        params['ifar'] = sfile[f'{ttype}/ifar_exc'.format(section)][:]
    else:
        raise KeyError("ifar or ifar_exc does not found in")
        
    bank_ids = sfile[f'{ttype}/template_id'.format(section)][:] #esto tambien tiene longitud 2060
    
    for ifo in ifos:
        params[f'time_{ifo}'] = sfile[f'{ttype}/{ifo}/time'][:] #{ifo} es variable que define el nombre del detector, /time es el dataset específico que se quiere extraer.
        #estas lineas es evaluan así: params['time_H1'] = sfile["found_after_vetoes/H1/time"][:]
        params[f'tid_{ifo}'] = sfile[f'{ttype}/{ifo}/trigger_id'][:]
        tids = sfile[f'{ttype}/{ifo}/trigger_id'][:]
        tids = tids[tids >= 0]
        #print('Estaba comentado y abajo imprime: ifo, tids.shape, params[tid_ifo].shape \n')
        #print(ifo, tids.shape, params[f'tid_{ifo}'].shape)
         # documentación dice: Provides easy access to the parameters of single-detector CBC triggers.
        sngls = pycbc.io.SingleDetTriggers(merge_trig_temp.format(ifo), bank_file,  #crea un objeto sngls usando la clase SingleDetTriggers de pycbc.io
                                None, None, None, detector=ifo, premask=tids)
        for field in merge_fields:
            print(f"reading field {field} for {ifo}")
            params[f'{field}_{ifo}'] = -1 * np.ones_like(params[f'time_{ifo}'])
            # print(field, f'{field}_{ifo}', params[f'{field}_{ifo}'].shape)
            # These are idexings and hence RAM heavy
             #crea un nuevo array que tiene la misma forma y tipo de datos que el array dado, pero con -1 por multiplicarlo por -1
            #esta linea la llama así: params[f'coa_phase_H1'] = -1 * np.ones_like(params[f'time_H1'])
            # print(field, f'{field}_{ifo}', params[f'{field}_{ifo}'].shape)
            # These are idexings and hence RAM heavy
            try: #usa una mascara boleana y seleciona del primer campos los que son True, hice un ejemplo y luego a esos valores les asigna que sale de esa clase
                params[f'{field}_{ifo}'][params[f'tid_{ifo}'] >= 0] = getattr(sngls, f'{field}')
            except AttributeError: #Si sngls no tiene un atributo llamado field, se produce un AttributeError
                # En ese caso, el código intenta obtener los datos usando sngls.get_column(f'{field}'), que es otra forma de acceder a una columna de datos en sngls.
                params[f'{field}_{ifo}'][params[f'tid_{ifo}'] >= 0] = sngls.get_column(f'{field}')
            except Exception as e:
                print(f"An error occurred: {e}")
     #coinc es algo que se importo de pycbc          
    #sacar el tiempo promedio de lo tres detectores, se evalua así la primera: coinc.mean_if_greater_than_zero([params[f'time_H1'][0] for ifo in ifos])[0]
    #la función mean_if_greater_than_zero saca una lista [], pero entonces se pone el [0] al final para que quede solo el numero, eso lo hace 2060 veces y luego
    #saca el promedio            
    params['mean_time'] = [coinc.mean_if_greater_than_zero([params[f'time_{ifo}'][cnt] for ifo in ifos])[0]
                           for cnt in range(len(params['stat']))]
    params['mean_time'] = np.array(params['mean_time'])

    template_fields = ['mass1', 'mass2', 'spin1z', 'spin2z']
    for field in template_fields:
        params[f'{field}'] = bank[f'{field}'][:][bank_ids]
    
    return params 

In [7]:
def LUCAS_get_trigger_info(ttype, sfile, bank, merge_trig_temp):
    '''Read all relevant information
    '''
    merge_fields = ['coa_phase', 'snr']
    #ifos toma el atributo ifos del archivo sfile y lo divide en una lista de interferómetros, que se usan para acceder a datos específicos de cada detector
    ifos = sfile.attrs['ifos'].split(' ') 
    
    params = {}
    params['stat'] = sfile[f'{ttype}/stat'.format(section)][:]
    if 'ifar' in sfile[f'{ttype}'.format(section)]:
        params['ifar'] = sfile[f'{ttype}/ifar'.format(section)][:]
    elif 'ifar_exc' in sfile[f'{ttype}'.format(section)]:
        params['ifar'] = sfile[f'{ttype}/ifar_exc'.format(section)][:]
    else:
        print(f"Advertencia: 'ifar' o 'ifar_exc' no encontrados en '{ttype}'")
        params['ifar'] = None  # Asigna None o un valor predeterminado si es necesario
        
    bank_ids = sfile[f'{ttype}/template_id'.format(section)][:]
    
    for ifo in ifos:
        params[f'time_{ifo}'] = sfile[f'{ttype}/{ifo}/time'][:]
        params[f'tid_{ifo}'] = sfile[f'{ttype}/{ifo}/trigger_id'][:]
        tids = sfile[f'{ttype}/{ifo}/trigger_id'][:]
        tids = tids[tids >= 0]
        # print(ifo, tids.shape, params[f'tid_{ifo}'].shape)
        
        sngls = pycbc.io.SingleDetTriggers(merge_trig_temp.format(ifo), bank_file, 
                                None, None, None, detector=ifo, premask=tids)
        for field in merge_fields:
            print(f"reading field {field} for {ifo}")
            params[f'{field}_{ifo}'] = -1 * np.ones_like(params[f'time_{ifo}'])
            # print(field, f'{field}_{ifo}', params[f'{field}_{ifo}'].shape)
            # These are idexings and hence RAM heavy
            try:
                params[f'{field}_{ifo}'][params[f'tid_{ifo}'] >= 0] = getattr(sngls, f'{field}')
            except AttributeError:
                params[f'{field}_{ifo}'][params[f'tid_{ifo}'] >= 0] = sngls.get_column(f'{field}')
            except Exception as e:
                print(f"An error occurred: {e}")
                
    params['mean_time'] = [coinc.mean_if_greater_than_zero([params[f'time_{ifo}'][cnt] for ifo in ifos])[0]
                           for cnt in range(len(params['stat']))]
    params['mean_time'] = np.array(params['mean_time'])

    template_fields = ['mass1', 'mass2', 'spin1z', 'spin2z']
    for field in template_fields:
        params[f'{field}'] = bank[f'{field}'][:][bank_ids]
    
    return params

## Paths on CIT for first 5 O3 chunks:
 - /home/bhooshan.gadre/work/O3/sub_solar_search/chunk{2,4,5}_rerun_extrainj_rerun/output/
 - /home/bhooshan.gadre/work/O3/sub_solar_search/chunk1_rerun_extrainj_ssm2/output
 - /home/bhooshan.gadre/work/O3/sub_solar_search/chunk3_rerun_extrainj/output/

## Injection sets:
 - There are 4 injection sets per chunk: SSM1SEOBNRV4 and SSM{1,2,3}STT5
 - Each can be read separately using the code below

In [8]:
run_path = '/home/bhooshan.gadre/work/O3/sub_solar_search/chunk5_rerun_extrainj_rerun/output'

## Use the following to figure out chunk start (UTC) and chunk duration (sec) to be used later to access relevant files
!ls $run_path/bank/H1L1V1-BANK2HDF-*

## output should be H1L1V1-BANK2HDF-{chunk_start}-{chunk_duration}.hdf

/home/bhooshan.gadre/work/O3/sub_solar_search/chunk5_rerun_extrainj_rerun/output/bank/H1L1V1-BANK2HDF-1241010950-714094.hdf


In [9]:
chunk_start = 1241010950
chunk_duration = 714094

## Use the following to read different injection sets.

## To read injected parameters:

In [10]:
inj_type = 'SSM1SEOBNRV4' ## SSM{1,2,3}STT5

sfile =  HFile(os.path.join(run_path, 
                            f'{inj_type}_INJ_coinc/H1L1V1-HDFINJFIND_{inj_type}_INJ_INJECTIONS-{chunk_start}-{chunk_duration}.hdf'))
## The file handle below contain all the injection sets combined with missed-found info. 
## But the file cannot be used to extract trigger template parameters.

# allinj = HFile(os.path.join(run_path, 
                            # f'allinj/H1L1V1-HDFINJFIND_ALL_INJECTIONS-{chunk_start}-{chunk_duration}.hdf'))

In [11]:
sfile['injections'].keys()

<KeysViewHDF5 ['coa_phase', 'distance', 'eff_dist_h', 'eff_dist_l', 'eff_dist_v', 'end_time', 'inclination', 'latitude', 'longitude', 'mass1', 'mass2', 'optimal_snr_H1', 'optimal_snr_L1', 'optimal_snr_V1', 'polarization', 'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z']>

In [12]:
found_ids = sfile['found_after_vetoes']['injection_index'][:]

found_injection = {k: sfile['injections'][k][:][found_ids] for k in sfile['injections'].keys()}
# found_injection

## To read found injection information
 - trigger template (same for all ifos due to exact coincidence search), stat, FAR, ifo specofic SNRs, phases, times


In [13]:
merge_trig_temp = os.path.join(run_path, 
                               f'{inj_type}_INJ_coinc/{{}}-HDF_TRIGGER_MERGE_{inj_type}_INJ_INJECTIONS-{chunk_start}-{chunk_duration}.hdf'
                              )

In [14]:
bank_file = os.path.join(run_path, 
                         f'bank/H1L1V1-BANK2HDF-{chunk_start}-{chunk_duration}.hdf')
bank = HFile(bank_file, 'r')

In [16]:
bank.keys()

<KeysViewHDF5 ['f_lower', 'mass1', 'mass2', 'spin1z', 'spin2z', 'template_duration', 'template_hash']>

In [17]:
bank['mass1']

<HDF5 dataset "mass1": shape (1864323,), type "<f8">

In [20]:
section = 'found_after_vetoes'
params_eob_inj = get_trigger_info(section, sfile, bank, merge_trig_temp)

# tid_{ifo} field is really irrelevant after fetching all the information. Those can/should be removed to save memory. 

reading field coa_phase for H1
reading field snr for H1
reading field coa_phase for L1
reading field snr for L1
reading field coa_phase for V1
reading field snr for V1


In [21]:
params_eob_inj.keys()

dict_keys(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1', 'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1', 'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z', 'spin2z'])

In [22]:
df_params_eob_inj = pd.DataFrame(params_eob_inj)
df_params_eob_inj

,stat,ifar,time_H1,tid_H1,coa_phase_H1,snr_H1,time_L1,tid_L1,coa_phase_L1,snr_L1,time_V1,tid_V1,coa_phase_V1,snr_V1,mean_time,mass1,mass2,spin1z,spin2z
0,30.140493,6.778316e+04,-1.000000e+00,-1,-1.000000,-1.000000,1.241022e+09,6413714,2.857998,9.802570,1.241022e+09,7715213,-1.405852,6.532456,1.241022e+09,6.837991,0.824298,0.525395,0.544641
1,-3.368488,6.082226e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241024e+09,2082826,-1.589477,5.103356,1.241024e+09,2501744,-0.780636,5.181249,1.241024e+09,8.278241,0.902386,-0.086355,-0.419087
2,405.313568,6.778316e+04,-1.000000e+00,-1,-1.000000,-1.000000,1.241025e+09,9840653,2.082934,253.506027,1.241025e+09,11842750,-0.149286,41.550201,1.241025e+09,7.226806,0.945996,-0.193048,-0.670380
3,328.160034,6.778316e+04,-1.000000e+00,-1,-1.000000,-1.000000,1.241029e+09,2201064,-0.385503,93.659904,1.241029e+09,2644291,-1.022927,25.238707,1.241029e+09,6.636675,0.766273,0.631528,0.441391
4,-3.869865,5.933771e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241030e+09,2913937,0.354279,5.485909,1.241030e+09,3502458,3.128220,5.161547,1.241030e+09,5.496568,0.558903,-0.793989,-0.418408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,-2.826163,2.184412e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241713e+09,1390226,0.303705,5.212824,1.241713e+09,1670044,1.401478,5.231639,1.241713e+09,8.854094,0.947527,-0.201915,-0.615814
2056,174.626312,1.283115e+04,1.241715e+09,8060542,-1.937861,24.658665,1.241715e+09,8996193,0.704603,15.782867,1.241715e+09,10825687,2.451597,7.314697,1.241715e+09,7.970858,0.886154,-0.075947,-0.674858
2057,18.031921,1.661769e-01,1.241720e+09,8085507,-1.950470,5.526212,1.241720e+09,9023723,1.204573,7.793991,-1.000000e+00,-1,-1.000000,-1.000000,1.241720e+09,4.902050,0.729935,-0.251312,-0.428933
2058,250.528412,1.283115e+04,1.241721e+09,7603540,1.779453,20.969648,1.241721e+09,8486039,-0.952633,30.388594,1.241721e+09,10211308,1.525583,12.525005,1.241721e+09,6.764315,0.924327,-0.758560,0.387254


In [23]:
params_eob_inj_lucas = LUCAS_get_trigger_info(section, sfile, bank, merge_trig_temp)

reading field coa_phase for H1
reading field snr for H1
reading field coa_phase for L1
reading field snr for L1
reading field coa_phase for V1
reading field snr for V1


In [24]:
params_eob_inj_lucas.keys()

dict_keys(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1', 'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1', 'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z', 'spin2z'])

In [25]:
df_params_eob_inj_lucas = pd.DataFrame(params_eob_inj_lucas)
df_params_eob_inj_lucas

,stat,ifar,time_H1,tid_H1,coa_phase_H1,snr_H1,time_L1,tid_L1,coa_phase_L1,snr_L1,time_V1,tid_V1,coa_phase_V1,snr_V1,mean_time,mass1,mass2,spin1z,spin2z
0,30.140493,6.778316e+04,-1.000000e+00,-1,-1.000000,-1.000000,1.241022e+09,6413714,2.857998,9.802570,1.241022e+09,7715213,-1.405852,6.532456,1.241022e+09,6.837991,0.824298,0.525395,0.544641
1,-3.368488,6.082226e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241024e+09,2082826,-1.589477,5.103356,1.241024e+09,2501744,-0.780636,5.181249,1.241024e+09,8.278241,0.902386,-0.086355,-0.419087
2,405.313568,6.778316e+04,-1.000000e+00,-1,-1.000000,-1.000000,1.241025e+09,9840653,2.082934,253.506027,1.241025e+09,11842750,-0.149286,41.550201,1.241025e+09,7.226806,0.945996,-0.193048,-0.670380
3,328.160034,6.778316e+04,-1.000000e+00,-1,-1.000000,-1.000000,1.241029e+09,2201064,-0.385503,93.659904,1.241029e+09,2644291,-1.022927,25.238707,1.241029e+09,6.636675,0.766273,0.631528,0.441391
4,-3.869865,5.933771e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241030e+09,2913937,0.354279,5.485909,1.241030e+09,3502458,3.128220,5.161547,1.241030e+09,5.496568,0.558903,-0.793989,-0.418408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,-2.826163,2.184412e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241713e+09,1390226,0.303705,5.212824,1.241713e+09,1670044,1.401478,5.231639,1.241713e+09,8.854094,0.947527,-0.201915,-0.615814
2056,174.626312,1.283115e+04,1.241715e+09,8060542,-1.937861,24.658665,1.241715e+09,8996193,0.704603,15.782867,1.241715e+09,10825687,2.451597,7.314697,1.241715e+09,7.970858,0.886154,-0.075947,-0.674858
2057,18.031921,1.661769e-01,1.241720e+09,8085507,-1.950470,5.526212,1.241720e+09,9023723,1.204573,7.793991,-1.000000e+00,-1,-1.000000,-1.000000,1.241720e+09,4.902050,0.729935,-0.251312,-0.428933
2058,250.528412,1.283115e+04,1.241721e+09,7603540,1.779453,20.969648,1.241721e+09,8486039,-0.952633,30.388594,1.241721e+09,10211308,1.525583,12.525005,1.241721e+09,6.764315,0.924327,-0.758560,0.387254


## Use the following to read full_data (real data foreground and background)

In [31]:
bank_file = os.path.join(run_path, 
                         f'bank/H1L1V1-BANK2HDF-{chunk_start}-{chunk_duration}.hdf') #es la misma que la de arriba 
merge_trig_temp = os.path.join(run_path, 
                               f'full_data/{{}}-HDF_TRIGGER_MERGE_FULL_DATA-{chunk_start}-{chunk_duration}.hdf')
statmap_file = os.path.join(run_path, 
                            f'full_data/H1L1V1-COMBINE_STATMAP_FULL_DATA-{chunk_start}-{chunk_duration}.hdf')

In [32]:
bank = HFile(bank_file, 'r')
sfile = HFile(statmap_file, 'r')

In [35]:
section = 'foreground' # 'background', 'background_exc'
paramsf = get_trigger_info(section, sfile, bank, merge_trig_temp)

reading field coa_phase for H1
reading field snr for H1
reading field coa_phase for L1
reading field snr for L1
reading field coa_phase for V1
reading field snr for V1


In [36]:
paramsf.keys()

dict_keys(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1', 'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1', 'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z', 'spin2z'])

In [40]:
df_paramsf = pd.DataFrame(paramsf)
df_paramsf

,stat,ifar,time_H1,tid_H1,coa_phase_H1,snr_H1,time_L1,tid_L1,coa_phase_L1,snr_L1,time_V1,tid_V1,coa_phase_V1,snr_V1,mean_time,mass1,mass2,spin1z,spin2z
0,-3.392430,6.884099e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,1673647446,-2.942494,5.928938,1.241014e+09,2008550684,-1.628540,5.326768,1.241014e+09,7.115521,0.852935,-0.126245,0.400083
1,-5.640567,6.565949e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,1171055633,-2.975821,4.501229,1.241014e+09,1405381987,2.798257,5.198503,1.241014e+09,0.672526,0.242774,0.669515,0.085895
2,-5.329537,6.583738e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,2406862606,-0.037575,4.518660,1.241014e+09,2888447001,2.481364,5.279398,1.241014e+09,1.453767,0.211166,-0.730410,-0.065974
3,-2.302806,7.419983e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,22963221,-2.735272,5.108595,1.241014e+09,27557328,0.108963,5.414484,1.241014e+09,1.484124,0.513304,-0.154248,0.018412
4,-1.974897,7.694578e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,2440766033,0.311613,5.612871,1.241014e+09,2929131002,-1.683660,4.812182,1.241014e+09,1.745579,0.218740,-0.671804,0.004558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31982,-2.643446,7.327394e-07,1.241725e+09,1832030467,-2.932472,5.146532,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,2454247430,0.883326,4.975547,1.241725e+09,0.281268,0.281268,0.099380,0.099008
31983,-3.857190,6.827555e-07,1.241725e+09,1732910455,1.957487,5.287742,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,2321491641,0.522092,4.917641,1.241725e+09,0.203239,0.203239,0.090866,0.097425
31984,3.966397,1.556139e-05,1.241725e+09,677756983,-0.106051,6.088523,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,907887387,-2.986848,5.243014,1.241725e+09,1.486490,0.224822,-0.098460,-0.069973
31985,1.396329,2.561485e-06,1.241725e+09,1170487712,-0.250431,4.793497,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,1567970793,1.435919,6.063501,1.241725e+09,0.720593,0.225870,-0.063789,0.005885


In [41]:
section = 'foreground' # 'background', 'background_exc'
paramsf_lucas = LUCAS_get_trigger_info(section, sfile, bank, merge_trig_temp)

reading field coa_phase for H1
reading field snr for H1
reading field coa_phase for L1
reading field snr for L1
reading field coa_phase for V1
reading field snr for V1


In [42]:
paramsf_lucas.keys()

dict_keys(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1', 'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1', 'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z', 'spin2z'])

In [44]:
df_paramsf_lucas = pd.DataFrame(paramsf_lucas)
df_paramsf_lucas

,stat,ifar,time_H1,tid_H1,coa_phase_H1,snr_H1,time_L1,tid_L1,coa_phase_L1,snr_L1,time_V1,tid_V1,coa_phase_V1,snr_V1,mean_time,mass1,mass2,spin1z,spin2z
0,-3.392430,6.884099e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,1673647446,-2.942494,5.928938,1.241014e+09,2008550684,-1.628540,5.326768,1.241014e+09,7.115521,0.852935,-0.126245,0.400083
1,-5.640567,6.565949e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,1171055633,-2.975821,4.501229,1.241014e+09,1405381987,2.798257,5.198503,1.241014e+09,0.672526,0.242774,0.669515,0.085895
2,-5.329537,6.583738e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,2406862606,-0.037575,4.518660,1.241014e+09,2888447001,2.481364,5.279398,1.241014e+09,1.453767,0.211166,-0.730410,-0.065974
3,-2.302806,7.419983e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,22963221,-2.735272,5.108595,1.241014e+09,27557328,0.108963,5.414484,1.241014e+09,1.484124,0.513304,-0.154248,0.018412
4,-1.974897,7.694578e-07,-1.000000e+00,-1,-1.000000,-1.000000,1.241014e+09,2440766033,0.311613,5.612871,1.241014e+09,2929131002,-1.683660,4.812182,1.241014e+09,1.745579,0.218740,-0.671804,0.004558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31982,-2.643446,7.327394e-07,1.241725e+09,1832030467,-2.932472,5.146532,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,2454247430,0.883326,4.975547,1.241725e+09,0.281268,0.281268,0.099380,0.099008
31983,-3.857190,6.827555e-07,1.241725e+09,1732910455,1.957487,5.287742,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,2321491641,0.522092,4.917641,1.241725e+09,0.203239,0.203239,0.090866,0.097425
31984,3.966397,1.556139e-05,1.241725e+09,677756983,-0.106051,6.088523,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,907887387,-2.986848,5.243014,1.241725e+09,1.486490,0.224822,-0.098460,-0.069973
31985,1.396329,2.561485e-06,1.241725e+09,1170487712,-0.250431,4.793497,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,1567970793,1.435919,6.063501,1.241725e+09,0.720593,0.225870,-0.063789,0.005885


In [24]:
paramsf

# tid_{ifo} field is really irrelevant after fetching all the information. Those can/should be removed to save memory.

{'stat': array([-3.3924296 , -5.640567  , -5.3295374 , ...,  3.966397  ,
         1.3963292 , -0.74588513], dtype=float32),
 'ifar': array([6.88409937e-07, 6.56594854e-07, 6.58373804e-07, ...,
        1.55613850e-05, 2.56148508e-06, 1.01593787e-06]),
 'time_H1': array([-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
         1.24172479e+09,  1.24172482e+09,  1.24172483e+09]),
 'tid_H1': array([        -1,         -1,         -1, ...,  677756983, 1170487712,
        1787500513]),
 'coa_phase_H1': array([-1.        , -1.        , -1.        , ..., -0.10605115,
        -0.25043085, -0.62821639]),
 'snr_H1': array([-1.        , -1.        , -1.        , ...,  6.08852339,
         4.79349661,  5.447474  ]),
 'time_L1': array([ 1.24101397e+09,  1.24101399e+09,  1.24101400e+09, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]),
 'tid_L1': array([1673647446, 1171055633, 2406862606, ...,         -1,         -1,
                -1]),
 'coa_phase_L1': array([-2.94249439, -2

In [25]:
paramsf.keys()

dict_keys(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1', 'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1', 'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z', 'spin2z'])

In [53]:
section = 'background'
paramsb = get_trigger_info(section, sfile, bank, merge_trig_temp)

KeyError: 'ifar or ifar_exc does not found in'

In [21]:
section = 'background'
paramsb = LUCAS_get_trigger_info(section, sfile, bank, merge_trig_temp)

Advertencia: 'ifar' o 'ifar_exc' no encontrados en 'background'
reading field coa_phase for H1
reading field snr for H1
reading field coa_phase for L1
reading field snr for L1
reading field coa_phase for V1
reading field snr for V1


In [25]:
df_b_lucas.keys()

Index(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1',
       'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1',
       'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z',
       'spin2z'],
      dtype='object')

In [24]:
df_b_lucas = pd.DataFrame(paramsb)

df_b_lucas.T

,0,1,2,3,4,5,6,7,8,9,...,86376739,86376740,86376741,86376742,86376743,86376744,86376745,86376746,86376747,86376748
stat,11.294158,9.403015,15.353808,10.206897,9.654624,6.049109,7.182386,10.603486,15.030932,11.323716,...,1.876716,0.580814,0.745328,3.094034,5.090391,10.505274,5.755412,10.592489,5.78455,6.391989
ifar,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
time_H1,1241068940.3125,1241069039.653809,1241069040.451172,1241068929.56543,1241068934.321777,1241069072.577148,1241069098.930664,1241069034.814941,1241068956.591797,1241069105.689453,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
tid_H1,1127235331,771126882,453317800,2171455454,580025962,1383242060,915494066,397672797,63944025,264121332,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
coa_phase_H1,0.834842,-2.499064,2.26294,0.95525,-0.938532,1.79763,2.861605,2.68003,-2.050329,-1.310181,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
snr_H1,5.409212,5.989699,6.200522,5.503916,5.275168,5.473706,5.253852,5.50472,5.287185,5.625994,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
time_L1,1241724526.314941,1241724599.645996,1241724569.642578,1241724431.563477,1241724434.322754,1241724572.581543,1241724598.929199,1241724522.8125,1241724424.996094,1241724571.682129,...,1241724247.419434,1241724264.559082,1241724317.969238,1241724477.212891,1241724236.297363,1241723748.721191,1241724584.910156,1241724162.170898,1241724026.512695,1241724547.962402
tid_L1,1258292200,860761302,506018429,2423973226,647456251,1544049670,1021931340,443878308,71384651,294833614,...,2435346045,1248990255,373603106,1812449302,2346120735,1701856709,2336628174,947943049,1895338325,684372685
coa_phase_L1,-1.716748,0.70133,-1.28045,-1.359722,2.201225,-1.520956,-0.240703,-0.506534,1.406701,2.350275,...,-2.337708,-2.480454,-0.670138,0.178865,0.986888,-2.733922,1.87232,3.130712,1.157853,-1.541533
snr_L1,6.372437,5.488677,6.450004,6.292346,5.758871,5.419576,5.507119,6.159497,6.97239,6.255848,...,5.55159,5.476921,5.374461,5.502933,5.635464,7.160771,5.638478,6.231894,5.680794,5.961353


In [26]:
df_b_lucas

,stat,ifar,time_H1,tid_H1,coa_phase_H1,snr_H1,time_L1,tid_L1,coa_phase_L1,snr_L1,time_V1,tid_V1,coa_phase_V1,snr_V1,mean_time,mass1,mass2,spin1z,spin2z
0,11.294158,None,1.241069e+09,1127235331,0.834842,5.409212,1.241725e+09,1258292200,-1.716748,6.372437,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,2.771800,0.310002,-0.758829,0.041250
1,9.403015,None,1.241069e+09,771126882,-2.499064,5.989699,1.241725e+09,860761302,0.701330,5.488677,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,0.824191,0.236069,-0.614635,0.006864
2,15.353808,None,1.241069e+09,453317800,2.262940,6.200522,1.241725e+09,506018429,-1.280450,6.450004,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.548248,0.245045,0.881828,0.002887
3,10.206897,None,1.241069e+09,2171455454,0.955250,5.503916,1.241724e+09,2423973226,-1.359722,6.292346,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.926657,0.204499,-0.879955,0.052006
4,9.654624,None,1.241069e+09,580025962,-0.938532,5.275168,1.241724e+09,647456251,2.201225,5.758871,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.065197,0.363016,-0.375022,-0.001943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86376744,10.505274,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,1701856709,-2.733922,7.160771,1.241014e+09,2042394492,1.035929,5.434130,1.241369e+09,1.054185,0.212007,-0.725758,0.036721
86376745,5.755412,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241725e+09,2336628174,1.872320,5.638478,1.241015e+09,2804185946,-1.014992,6.343738,1.241370e+09,0.515853,0.214150,-0.611320,-0.087634
86376746,10.592489,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,947943049,3.130712,6.231894,1.241015e+09,1137571259,-0.869140,6.245370,1.241369e+09,0.431449,0.220175,-0.054366,0.071771
86376747,5.784550,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,1895338325,1.157853,5.680794,1.241014e+09,2274598369,2.258872,6.579111,1.241369e+09,1.046319,0.821623,0.697675,0.896271


In [27]:
# Guardar el DataFrame en un archivo txt
df_b_lucas.to_csv('df_b_lucas.txt', sep='\t', index=False)

In [54]:
#background_exc
DataFrametion = 'background_exc'
paramsb_exc = get_trigger_info(section, sfile, bank, merge_trig_temp)

KeyError: 'ifar or ifar_exc does not found in'

In [55]:
paramsb_exc.keys()

dict_keys(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1', 'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1', 'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z', 'spin2z'])

In [56]:
df_paramsb_exc = pd.DataFrame(paramsb_exc)

df_paramsb_exc

,stat,ifar,time_H1,tid_H1,coa_phase_H1,snr_H1,time_L1,tid_L1,coa_phase_L1,snr_L1,time_V1,tid_V1,coa_phase_V1,snr_V1,mean_time,mass1,mass2,spin1z,spin2z
0,11.294158,None,1.241069e+09,1127235331,0.834842,5.409212,1.241725e+09,1258292200,-1.716748,6.372437,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,2.771800,0.310002,-0.758829,0.041250
1,9.403015,None,1.241069e+09,771126882,-2.499064,5.989699,1.241725e+09,860761302,0.701330,5.488677,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,0.824191,0.236069,-0.614635,0.006864
2,15.353808,None,1.241069e+09,453317800,2.262940,6.200522,1.241725e+09,506018429,-1.280450,6.450004,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.548248,0.245045,0.881828,0.002887
3,10.206897,None,1.241069e+09,2171455454,0.955250,5.503916,1.241724e+09,2423973226,-1.359722,6.292346,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.926657,0.204499,-0.879955,0.052006
4,9.654624,None,1.241069e+09,580025962,-0.938532,5.275168,1.241724e+09,647456251,2.201225,5.758871,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.065197,0.363016,-0.375022,-0.001943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80182825,5.090391,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,2346120735,0.986888,5.635464,1.241015e+09,2815572736,-0.843690,6.533869,1.241370e+09,0.414613,0.213767,-0.018763,0.080074
80182826,10.505274,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,1701856709,-2.733922,7.160771,1.241014e+09,2042394492,1.035929,5.434130,1.241369e+09,1.054185,0.212007,-0.725758,0.036721
80182827,10.592489,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,947943049,3.130712,6.231894,1.241015e+09,1137571259,-0.869140,6.245370,1.241369e+09,0.431449,0.220175,-0.054366,0.071771
80182828,5.784550,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,1895338325,1.157853,5.680794,1.241014e+09,2274598369,2.258872,6.579111,1.241369e+09,1.046319,0.821623,0.697675,0.896271


In [48]:
#background_exc

section = 'background_exc'
paramsb_exc = LUCAS_get_trigger_info(section, sfile, bank, merge_trig_temp)

Advertencia: 'ifar' o 'ifar_exc' no encontrados en 'background_exc'
reading field coa_phase for H1
reading field snr for H1
reading field coa_phase for L1
reading field snr for L1
reading field coa_phase for V1
reading field snr for V1


In [49]:
paramsb_exc.keys()

dict_keys(['stat', 'ifar', 'time_H1', 'tid_H1', 'coa_phase_H1', 'snr_H1', 'time_L1', 'tid_L1', 'coa_phase_L1', 'snr_L1', 'time_V1', 'tid_V1', 'coa_phase_V1', 'snr_V1', 'mean_time', 'mass1', 'mass2', 'spin1z', 'spin2z'])

In [50]:
df_b_exc_lucas = pd.DataFrame(paramsb_exc)

df_b_exc_lucas

,stat,ifar,time_H1,tid_H1,coa_phase_H1,snr_H1,time_L1,tid_L1,coa_phase_L1,snr_L1,time_V1,tid_V1,coa_phase_V1,snr_V1,mean_time,mass1,mass2,spin1z,spin2z
0,11.294158,None,1.241069e+09,1127235331,0.834842,5.409212,1.241725e+09,1258292200,-1.716748,6.372437,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,2.771800,0.310002,-0.758829,0.041250
1,9.403015,None,1.241069e+09,771126882,-2.499064,5.989699,1.241725e+09,860761302,0.701330,5.488677,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,0.824191,0.236069,-0.614635,0.006864
2,15.353808,None,1.241069e+09,453317800,2.262940,6.200522,1.241725e+09,506018429,-1.280450,6.450004,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.548248,0.245045,0.881828,0.002887
3,10.206897,None,1.241069e+09,2171455454,0.955250,5.503916,1.241724e+09,2423973226,-1.359722,6.292346,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.926657,0.204499,-0.879955,0.052006
4,9.654624,None,1.241069e+09,580025962,-0.938532,5.275168,1.241724e+09,647456251,2.201225,5.758871,-1.000000e+00,-1,-1.000000,-1.000000,1.241397e+09,1.065197,0.363016,-0.375022,-0.001943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80182825,5.090391,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,2346120735,0.986888,5.635464,1.241015e+09,2815572736,-0.843690,6.533869,1.241370e+09,0.414613,0.213767,-0.018763,0.080074
80182826,10.505274,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,1701856709,-2.733922,7.160771,1.241014e+09,2042394492,1.035929,5.434130,1.241369e+09,1.054185,0.212007,-0.725758,0.036721
80182827,10.592489,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,947943049,3.130712,6.231894,1.241015e+09,1137571259,-0.869140,6.245370,1.241369e+09,0.431449,0.220175,-0.054366,0.071771
80182828,5.784550,None,-1.000000e+00,-1,-1.000000,-1.000000,1.241724e+09,1895338325,1.157853,5.680794,1.241014e+09,2274598369,2.258872,6.579111,1.241369e+09,1.046319,0.821623,0.697675,0.896271


In [51]:
# Guardar el DataFrame en un archivo txt
df_b_exc_lucas.to_csv('df_b_exc_lucas.txt', sep='\t', index=False)

In [52]:
print('h')

h
